# It's a Machine and Natural Language Tagger

In [1]:
from src.IaMaN.base import LM
from src.utils.data import load_ud
from src.utils.munge import stick_spaces
from pprint import pprint as pprint
from collections import defaultdict
from collections import Counter
from tqdm import tqdm
import numpy as np
import os, re

seed = 691; max_char = 200_000_000
m = 10; space = True; fine_tune = False; num_posttrain = 0; noise = 0.001
positional = 'dependent'; positionally_encode = True; bits = 50; update = True; btype = 'f'; ms_init = 'waiting_time'
runners = 10; gpu = False; tokenizer = 'hr-bpe'; decode_method = 'argmax'

print("Loading post-training data...")
posttrain_path = '/data/newstweet/week_2019-40_article_texts/'
total_posttrain = len([posttrain_file for posttrain_file in os.listdir(posttrain_path) if re.search("^\d+.txt$", posttrain_file)])
all_posttrain_files = [posttrain_file for posttrain_file in os.listdir(posttrain_path) if re.search("^\d+.txt$", posttrain_file)]
if num_posttrain:
    np.random.seed(seed)
    posttrain_files = np.random.choice(all_posttrain_files, size=num_posttrain, replace=False)
else:
    posttrain_files = np.array([])
ptdocs = [[[open(posttrain_path+posttrain_file).read()]] for posttrain_file in tqdm(posttrain_files)]
print("Loading gold-tagged UDs data...")
load_set = "GUM"
all_docs = load_ud("English", num_articles = 0, seed = seed, load_set = load_set, rebuild = True, space = space)
test_docs = [doc for doc in all_docs if 'test' in doc['id'] and len(doc['text']) <= max_char] # [:2]
train_docs = [doc for doc in all_docs if 'test' not in doc['id'] and len(doc['text']) <= max_char] # [:4]
nsamp = len(test_docs)
print('Avail. post-train, total post-train, Avail. gold, total gold-train, total test-gold: ', 
      total_posttrain, len(ptdocs), len(all_docs), len(train_docs), len(test_docs))

Loading post-training data...


0it [00:00, ?it/s]

Loading gold-tagged UDs data...


Avail. post-train, total post-train, Avail. gold, total gold-train, total test-gold:  14198 0 150 132 18


In [2]:
docs = [["".join([row[1] for row in s]) for s in d['conllu']] for d in train_docs]
tdocs = [["".join([row[1] for row in s]) for s in d['conllu']] for d in test_docs]
covering = [[[row[1] for row in s] for s in d['conllu']] for d in train_docs]
tcovering = [[[row[1] for row in s] for s in d['conllu']] for d in test_docs]
if not space:
    for d_i, d in enumerate(covering):
        for s_i, s in enumerate(d):
            covering[d_i][s_i] = stick_spaces(s)
    for d_i, d in enumerate(tcovering):
        for s_i, s in enumerate(d):
            tcovering[d_i][s_i] = stick_spaces(s)
covering_vocab = set([t for d in covering for s in d for t in s])

# 'lem': [[row[2] for row in s] for s in d['conllu']], # note: for speed, remove lemma layer
train_layers = {d_i: {'sty': [[d['s_type'][s_i] for row in s] for s_i, s in enumerate(d['conllu'])], 
                      'pos': [[row[3] for row in s] for s in d['conllu']], 
                      'sup': [[(str(int(row[6]) - int(row[0])) if int(row[6]) else row[6]) for row in s] for s in d['conllu']], 
                      'dep': [[row[7] for row in s] for s in d['conllu']]}
                for d_i, d in enumerate(train_docs)}
# 'lem': [[row[2] for row in s] for s in d['conllu']], # note: for speed, remove lemma layer
test_layers = {d_i: {'sty': [[d['s_type'][s_i] for row in s] for s_i, s in enumerate(d['conllu'])], 
                     'pos': [[row[3] for row in s] for s in d['conllu']], 
                     'sup': [[(str(int(row[6]) - int(row[0])) if int(row[6]) else row[6]) for row in s] for s in d['conllu']], 
                     'dep': [[row[7] for row in s] for s in d['conllu']]}
               for d_i, d in enumerate(test_docs)}

model = LM(m = m, tokenizer = tokenizer, noise = noise, seed = seed, space = space, positional = positional,
           positionally_encode = positionally_encode, runners = runners, gpu = gpu, bits = bits, 
           btype = btype, ms_init = ms_init)
data_streams = model.fit(docs, f'{load_set}-{nsamp}', covering = covering, all_layers = train_layers, fine_tune = fine_tune)

Training tokenizer...


Fitting:  20%|██        | 20/100 [00:42<02:49,  2.12s/it]


Built a vocabulary of 15327 types
Tokenizing documents...


100%|██████████| 132/132 [00:14<00:00,  8.85it/s]


Pre-processing data...


...

Building dense output heads...


100%|██████████| 23/23 [00:00<00:00, 32.55it/s]


Building transition matrices for tag-sequence decoding...


  0%|          | 0/7 [00:00<?, ?it/s]/code/IaMaN/src/IaMaN/base.py:613: RuntimeWarning: invalid value encountered in true_divide
  self._trXs[ltype] = (lambda M: M/M.sum(axis = 1)[:,None])(self._trXs[ltype])
100%|██████████| 7/7 [00:00<00:00, 2147.94it/s]

Done.
Model params, types, encoding size, contexts, vec dim, max sent, and % capacity used: 886984 10568 50 1071 353 258 9.681


In [3]:
model._ms

{'sty': 2,
 'pos': 6,
 'sup': 7,
 'dep': 7,
 'nov': 2,
 'iat': 2,
 'bot': 2,
 'eot': 2,
 'eos': 2,
 'eod': 2,
 'form': 8,
 'bits': 10}

In [4]:
model.interpret([[' Gone again!']], seed = seed)
for doc in model._documents:
    print('opening next doc:')
    for s in doc._sentences:
        print(f'opening next sent: {s._sty}')
        for t in s._tokens:
            print(f'opening next token: {t._form}, {t._lem}, {t._sep}, {t._pos}, {t._sup}, {t._dep}')
            print(["\""+w._form+"\"" for w in t._whatevers])

Tokenizing documents...


100%|██████████| 1/1 [00:00<00:00, 2425.86it/s]


Pre-processing data...


100%|██████████| 1/1 [00:00<00:00, 1569.72it/s]


Interpreting documents...


100%|██████████| 1/1 [00:00<00:00, 114.47it/s]


opening next doc:
opening next sent: decl
opening next token:  , None, False, SPACE, 1, space
['" "']
opening next token: Gone, None, False, PROPN, 0, root
['"Gon"', '"e"']
opening next token:  , None, False, SPACE, -1, space
['" "']
opening next token: again, None, False, X, -1, dep
['"again"']
opening next token: !, None, True, PUNCT, -1, punct
['"!"']


In [5]:
d_i = 3; s_max = 2
interpret_docs = list([docs[d_i][:s_max]])
print(interpret_docs)
model.interpret(interpret_docs, eval_layers = {0: {tag: train_layers[d_i][tag][:s_max] for tag in train_layers[d_i]}},
                eval_covering = [covering[d_i][:s_max]], seed = seed)

for doc in model._documents:
    print('opening next doc:')
    for s in doc._sentences:
        print(f'opening next sent: {s._sty}')
        for t in s._tokens:
            print(f'opening next token: {t._form}, {t._lem}, {t._sep}, {t._pos}, {t._sup}, {t._dep}')

[[' Emperor Norton ', ' Joshua Abraham Norton (c. 1818 – January 8, 1880), known as Emperor Norton, was a citizen of San Francisco, California, who in 1859 proclaimed himself "Norton I, Emperor of the United States". ']]
Tokenizing documents...


100%|██████████| 1/1 [00:00<00:00, 124.92it/s]


Pre-processing data...


100%|██████████| 1/1 [00:00<00:00, 181.36it/s]


Interpreting documents...


100%|██████████| 1/1 [00:00<00:00,  1.29it/s]

Document 0's Sentence segmentation performance:  {('F', 1.0), ('R', 1.0), ('P', 1.0)}
Document 0's STY accuracy:  0.5
Document 0's Token segmentation performance without space:  {('P', 0.9782608695652174), ('R', 0.9782608695652174), ('F', 0.9782608695652174)}
Document 0's POS accuracy with/out space: 0.7654320987654321 0.5777777777777777
Document 0's SUP:DEP accuracy with/out space:  0.5555555555555556 0.2

Overall sentence segmentation performance:  {('F', 1.0), ('R', 1.0), ('P', 1.0)}
Overall STY accuracy:  0.5
Overall Token segmentation performance without space:  {('P', 0.9782608695652174), ('R', 0.9782608695652174), ('F', 0.9782608695652174)}
Overall POS accuracy with/out space: 0.7654320987654321 0.5777777777777777
Overall SUP:DEP accuracy with/out space:  0.5555555555555556 0.2
opening next doc:
opening next sent: decl
opening next token:  , None, False, SPACE, 1, space
opening next token: Emperor, None, False, PROPN, 0, root
opening next token:  , None, False, SPACE, 1, space
o

In [6]:
d_i = 0; s_max = 1
interpret_docs = list([tdocs[d_i][:s_max]])
print(interpret_docs)
model.interpret(interpret_docs, eval_layers = {0: {tag: test_layers[d_i][tag][:s_max] for tag in test_layers[d_i]}},
                eval_covering = [tcovering[d_i][:s_max]], seed = seed)

for doc in model._documents:
    print('opening next doc:')
    for s in doc._sentences:
        print(f'opening next sent: {s._sty}')
        for t in s._tokens:
            print(f'opening next token: {t._form}, {t._lem}, {t._sep}, {t._pos}, {t._sup}, {t._dep}')

[[' The prevalence of discrimination across racial groups in contemporary America: ']]
Tokenizing documents...


100%|██████████| 1/1 [00:00<00:00, 325.64it/s]


Pre-processing data...


100%|██████████| 1/1 [00:00<00:00, 575.98it/s]


Interpreting documents...


100%|██████████| 1/1 [00:00<00:00, 14.49it/s]

Document 0's Sentence segmentation performance:  {('F', 1.0), ('R', 1.0), ('P', 1.0)}
Document 0's STY accuracy:  0.0
Document 0's Token segmentation performance without space:  {('F', 1.0), ('R', 1.0), ('P', 1.0)}
Document 0's POS accuracy with/out space: 0.7272727272727273 0.45454545454545453
Document 0's SUP:DEP accuracy with/out space:  0.5 0.09090909090909091

Overall sentence segmentation performance:  {('F', 1.0), ('R', 1.0), ('P', 1.0)}
Overall STY accuracy:  0.0
Overall Token segmentation performance without space:  {('F', 1.0), ('R', 1.0), ('P', 1.0)}
Overall POS accuracy with/out space: 0.7272727272727273 0.45454545454545453
Overall SUP:DEP accuracy with/out space:  0.5 0.09090909090909091
opening next doc:
opening next sent: decl
opening next token:  , None, False, SPACE, 1, space
opening next token: The, None, False, PROPN, 0, root
opening next token:  , None, False, SPACE, 1, space
opening next token: prevalence, None, False, PROPN, -2, flat
opening next token:  , None, F

In [7]:
d_i = 1; s_max = 2
interpret_docs = list([tdocs[d_i][:s_max]])
print(interpret_docs)
model.interpret(interpret_docs, eval_layers = {0: {tag: test_layers[d_i][tag][:s_max] for tag in test_layers[d_i]}},
                eval_covering = [tcovering[d_i][:s_max]], seed = seed)
for doc in model._documents:
    print('opening next doc:')
    for s in doc._sentences:
        print(f'opening next sent: {s._sty}')
        for t in s._tokens:
            print(f'opening next token: {t._form}, {t._lem}, {t._sep}, {t._pos}, {t._sup}, {t._dep}')

[[' 2. GUJJOLAAY EEGIMAA, ITS SPEAKERS AND THEIR NEIGHBOURS ', ' This section briefly presents the Gújjolaay Eegimaa (Eegimaa for short; Ethnologue code: ISO 639-3: bqj), its speakers and its varieties. ']]
Tokenizing documents...


100%|██████████| 1/1 [00:00<00:00, 194.40it/s]


Pre-processing data...


100%|██████████| 1/1 [00:00<00:00, 95.30it/s]


Interpreting documents...


100%|██████████| 1/1 [00:00<00:00,  1.49it/s]

Document 0's Sentence segmentation performance:  {('F', 1.0), ('R', 1.0), ('P', 1.0)}
Document 0's STY accuracy:  0.5
Document 0's Token segmentation performance without space:  {('P', 0.5507246376811594), ('R', 0.76), ('F', 0.638655462184874)}
Document 0's POS accuracy with/out space: 0.6666666666666666 0.3888888888888889
Document 0's SUP:DEP accuracy with/out space:  0.3484848484848485 0.027777777777777776

Overall sentence segmentation performance:  {('F', 1.0), ('R', 1.0), ('P', 1.0)}
Overall STY accuracy:  0.5
Overall Token segmentation performance without space:  {('P', 0.5507246376811594), ('R', 0.76), ('F', 0.638655462184874)}
Overall POS accuracy with/out space: 0.6666666666666666 0.3888888888888889
Overall SUP:DEP accuracy with/out space:  0.3484848484848485 0.027777777777777776
opening next doc:
opening next sent: decl
opening next token:  , None, False, SPACE, 1, space
opening next token: 2, None, False, PROPN, 5, vocative
opening next token: ., None, False, PROPN, 0, root


In [8]:
d_i = 1; s_max = 2
interpret_docs = list([tdocs[d_i][:s_max]])
print(interpret_docs)
model.interpret(interpret_docs, eval_layers = {0: {tag: test_layers[d_i][tag][:s_max] for tag in test_layers[d_i]}},
                covering = [tcovering[d_i][:s_max]], seed = seed)
for doc in model._documents:
    print('opening next doc:')
    for s in doc._sentences:
        print(f'opening next sent: {s._sty}')
        for t in s._tokens:
            print(f'opening next token: {t._form}, {t._lem}, {t._sep}, {t._pos}, {t._sup}, {t._dep}')

[[' 2. GUJJOLAAY EEGIMAA, ITS SPEAKERS AND THEIR NEIGHBOURS ', ' This section briefly presents the Gújjolaay Eegimaa (Eegimaa for short; Ethnologue code: ISO 639-3: bqj), its speakers and its varieties. ']]
Tokenizing documents...


100%|██████████| 1/1 [00:00<00:00, 218.20it/s]


Pre-processing data...


100%|██████████| 1/1 [00:00<00:00, 115.33it/s]


Interpreting documents...


100%|██████████| 1/1 [00:00<00:00,  2.75it/s]

Document 0's STY accuracy:  0.5
Document 0's POS accuracy with/out space: 0.7121212121212122 0.4722222222222222
Document 0's SUP:DEP accuracy with/out space:  0.36363636363636365 0.027777777777777776

Overall STY accuracy:  0.5
Overall POS accuracy with/out space: 0.7121212121212122 0.4722222222222222
Overall SUP:DEP accuracy with/out space:  0.36363636363636365 0.027777777777777776
opening next doc:
opening next sent: decl
opening next token:  , None, False, SPACE, 1, space
opening next token: 2., None, False, PROPN, 0, root
opening next token:  , None, False, SPACE, -1, space
opening next token: GUJJOLAAY, None, False, PROPN, 2, nsubj
opening next token:  , None, False, SPACE, 1, space
opening next token: EEGIMAA, None, False, VERB, -4, punct
opening next token: ,, None, False, PUNCT, -1, punct
opening next token:  , None, False, SPACE, 1, space
opening next token: ITS, None, False, VERB, -3, nsubj
opening next token:  , None, False, SPACE, -1, space
opening next token: SPEAKERS, Non

In [9]:
model.interpret(tdocs, eval_layers = test_layers, eval_covering = tcovering, seed = seed, verbose_result = False, decode_method = decode_method)

Tokenizing documents...


100%|██████████| 18/18 [00:01<00:00,  9.12it/s]


Pre-processing data...


100%|██████████| 18/18 [00:02<00:00,  7.21it/s]


Interpreting documents...


...

Document 15's Sentence segmentation performance:  {('F', 1.0), ('R', 1.0), ('P', 1.0)}
Document 15's STY accuracy:  0.8421052631578947
Document 15's Token segmentation performance without space:  {('P', 0.8428184281842819), ('R', 0.8735955056179775), ('F', 0.8579310344827586)}
Document 15's POS accuracy with/out space: 0.7687296416938111 0.5456
Document 15's SUP:DEP accuracy with/out space:  0.5749185667752443 0.1936



 94%|█████████▍| 17/18 [03:32<00:10, 10.99s/it]

Document 16's Sentence segmentation performance:  {('F', 1.0), ('R', 1.0), ('P', 1.0)}
Document 16's STY accuracy:  0.32558139534883723
Document 16's Token segmentation performance without space:  {('P', 0.9100968188105117), ('R', 0.9228611500701263), ('F', 0.9164345403899722)}
Document 16's POS accuracy with/out space: 0.774781919111816 0.5672333848531684
Document 16's SUP:DEP accuracy with/out space:  0.5900079302141158 0.2241112828438949



100%|██████████| 18/18 [03:41<00:00, 12.31s/it]

Document 17's Sentence segmentation performance:  {('F', 1.0), ('R', 1.0), ('P', 1.0)}
Document 17's STY accuracy:  0.4153846153846154
Document 17's Token segmentation performance without space:  {('P', 0.9175068744271311), ('R', 0.9242843951985226), ('F', 0.9208831646734131)}
Document 17's POS accuracy with/out space: 0.7970118495620814 0.6044176706827309
Document 17's SUP:DEP accuracy with/out space:  0.5744461617722824 0.1897590361445783

Overall sentence segmentation performance:  {('F', 1.0), ('R', 1.0), ('P', 1.0)}
Overall STY accuracy:  0.6588366890380313
Overall Token segmentation performance without space:  {('P', 0.8734375845462908), ('R', 0.895683054045056), ('F', 0.8844204585924444)}
Overall POS accuracy with/out space: 0.7636820730999807 0.5501356684755797
Overall SUP:DEP accuracy with/out space:  0.5603687230065106 0.18296743956586087


In [10]:
model.interpret(tdocs, eval_layers = test_layers, covering = tcovering, seed = seed, verbose_result = False, decode_method = decode_method)

Tokenizing documents...


100%|██████████| 18/18 [00:01<00:00,  9.21it/s]


Pre-processing data...


100%|██████████| 18/18 [00:02<00:00,  7.68it/s]


Interpreting documents...


...

Document 15's STY accuracy:  0.8421052631578947
Document 15's POS accuracy with/out space: 0.8127035830618893 0.632
Document 15's SUP:DEP accuracy with/out space:  0.5724755700325733 0.1888



 94%|█████████▍| 17/18 [03:02<00:09,  9.66s/it]

Document 16's STY accuracy:  0.32558139534883723
Document 16's POS accuracy with/out space: 0.7930214115781126 0.6027820710973725
Document 16's SUP:DEP accuracy with/out space:  0.591593973037272 0.2302936630602782



100%|██████████| 18/18 [03:10<00:00, 10.61s/it]

Document 17's STY accuracy:  0.4153846153846154
Document 17's POS accuracy with/out space: 0.824317362184441 0.6576305220883534
Document 17's SUP:DEP accuracy with/out space:  0.583204533745492 0.20983935742971888

Overall STY accuracy:  0.6588366890380313
Overall POS accuracy with/out space: 0.7885966608650808 0.5978046373951653
Overall SUP:DEP accuracy with/out space:  0.5659124605169857 0.19394425259003453


In [11]:
model.post_train(ptdocs, update = update, fine_tune = fine_tune, bits = bits)
if fine_tune and ptdocs:
    model.fine_tune(docs, covering = covering, all_layers = train_layers, streams = data_streams)

In [12]:
if ptdocs:
    model.interpret(tdocs, eval_layers = test_layers, eval_covering = tcovering, seed = seed, verbose_result = False, decode_method = decode_method)

In [13]:
if ptdocs:
    model.interpret(tdocs, eval_layers = test_layers, covering = tcovering, seed = seed, verbose_result = False, decode_method = decode_method)